In [435]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline



### The below cell contains two functions that take inputs `qx` or `kx` and return transformed `qx` or transformed `kx`. The below cell is the easiest way to understand about what goes on in `rotary positional embedding` without much going into mathematics.

In [ ]:
"""How we estimate the transformation
"""

def Compute_angles(head_dim:int, seq_len:int, device:str = 'cpu', theta:int = 10000)->torch.tensor:

  theta_numerator = torch.arange(0, head_dim, 2).float(); '(hd // 2, )'
  theta = 1.0 / (theta ** (theta_numerator / head_dim)).to(device); '(hd // 2, )'


  m = torch.arange(seq_len).to(device); "(T, )"
  angles = torch.outer(m, theta); '(T, hd // 2)'

  return angles.float()

def apply_rotation(x:torch.tensor, angles:torch.tensor):
  B, T, nh, hd = x.shape
  x_re = x.reshape(B, T, nh, hd // 2, 2);'(B, T, nh, hd) ---> (B, T, nh, hd // 2, 2)'

  angles = angles.unsqueeze(0).unsqueeze(2); '(T, hd // 2) ---> (1, T, 1, hd // 2)'
  angles = angles.repeat(B, 1, nh, 1); '(1, T, 1, hd // 2) ---> (B, T, nh, hd // 2)'



  cos_angles = torch.cos(angles); '(B, T, nh, hd // 2)'

  sin_angles = torch.sin(angles); '(B, T, nh, hd // 2)'

  cos_angles = cos_angles.unsqueeze(4).unsqueeze(5); '(B, T, nh, hd // 2) ---> (B, T, nh, hd // 2, 1, 1)'
  sin_angles = sin_angles.unsqueeze(4).unsqueeze(5); '(B, T, nh, hd // 2) ---> (B, T, nh, hd // 2, 1, 1)'

  cos_sin1 = torch.cat((cos_angles, -1 * sin_angles), -1); '(B, T, nh, hd // 2, 1, 1) ---> (B, T, nh, hd // 2, 1, 2)'

  cos_sin2 = torch.cat((sin_angles, cos_angles), -1); '(B, T, nh, hd // 2, 1, 1) ---> (B, T, nh, hd // 2, 1, 2)'

  rotation_matrix = torch.cat((cos_sin1, cos_sin2), dim = -2); '(B, T, nh, hd // 2, 1, 2) ---> (B, T, nh, hd // 2, 2, 2)'

  x_re = x_re.unsqueeze(5); '(B, T, nh, hd // 2, 2) ---> (B, T, nh, hd // 2, 2, 1)'

  x_rotated = rotation_matrix @ x_re; '(B, T, nh, hd // 2, 2, 2) --> (B, T, nh, hd // 2, 2, 1)'

  x_rotated = x_rotated.squeeze(); '(B, T, nh, hd // 2, 2, 1) ---> (B, T, nh, hd // 2, 2)'

  x_rotated = x_rotated.reshape(B, T, nh, hd); '(B, T, nh, hd // 2, 2) ---> (B, T, nh, hd)'

  assert x_rotated.shape == x.shape; 'Shapes must be same'

  return x_rotated


#### The below cell contains two functions that take inputs `qx` or `kx` and return transformed `qx` or transformed `kx`. I took the code from `llamma2`.

In [ ]:
"""How they estimate the transformation
"""
from typing import Optional, Tuple

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    """
    Precompute the frequency tensor for complex exponentials (cis) with given dimensions.

    This function calculates a frequency tensor with complex exponentials using the given dimension 'dim'
    and the end index 'end'. The 'theta' parameter scales the frequencies.
    The returned tensor contains complex values in complex64 data type.

    Args:
        dim (int): Dimension of the frequency tensor.
        end (int): End index for precomputing frequencies.
        theta (float, optional): Scaling factor for frequency computation. Defaults to 10000.0.

    Returns:
        torch.Tensor: Precomputed frequency tensor with complex exponentials.




    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore

    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    return freqs_cis

def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    """
    Reshape frequency tensor for broadcasting it with another tensor.

    This function reshapes the frequency tensor to have the same shape as the target tensor 'x'
    for the purpose of broadcasting the frequency tensor during element-wise operations.

    Args:
        freqs_cis (torch.Tensor): Frequency tensor to be reshaped.
        x (torch.Tensor): Target tensor for broadcasting compatibility.

    Returns:
        torch.Tensor: Reshaped frequency tensor.

    Raises:
        AssertionError: If the frequency tensor doesn't match the expected shape.
        AssertionError: If the target tensor 'x' doesn't have the expected number of dimensions.
    """
    print(freqs_cis.shape, x.shape)
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(*shape)


def apply_rotary_emb(
    xq: torch.Tensor,
    freqs_cis: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Apply rotary embeddings to input tensors using the given frequency tensor.

    This function applies rotary embeddings to the given query 'xq' and key 'xk' tensors using the provided
    frequency tensor 'freqs_cis'. The input tensors are reshaped as complex numbers, and the frequency tensor
    is reshaped for broadcasting compatibility. The resulting tensors contain rotary embeddings and are
    returned as real tensors.

    Args:
        xq (torch.Tensor): Query tensor to apply rotary embeddings.
        xk (torch.Tensor): Key tensor to apply rotary embeddings.
        freqs_cis (torch.Tensor): Precomputed frequency tensor for complex exponentials.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Tuple of modified query tensor and key tensor with rotary embeddings.


    """
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq)

In [ ]:
B, T, C = 4, 16, 32 #B = Batch size, T = sequence length, C = embedding size(dim in Llamma2),

torch.manual_seed(1)
embedding = torch.randn((B, T, C))

q_weight = torch.randn((C, C))

k_weight = torch.randn((C, C))

qx = embedding @ q_weight #(B, T, C) @ (C, C) ---> (B, T, C) @ (B, C, C)---> (B, T, C)
kx = embedding @ k_weight #(B, T, C) @ (C, C) ---> (B, T, C) @ (B, C, C)---> (B, T, C)

nh = 4 #nh = number of head

qx = qx.reshape(B, T, nh, C // nh) #C // nh = hd
kx = kx.reshape(B, T, nh, C // nh)
qx.shape, kx.shape

#Comparison

In [ ]:
hs = C // nh #Head size or head dimension(hd)

angles_their = precompute_freqs_cis(hs, T)
rotated_qx_their = apply_rotary_emb(qx, angles_their)
print(angles_their.shape, qx.shape, rotated_qx_their.shape)

angles_ours = Compute_angles(hs, T)
rotated_qx_ours = apply_rotation(qx, angles_ours)
print(angles_ours.shape, qx.shape, rotated_qx_ours.shape)

print(torch.allclose(rotated_qx_their, rotated_qx_ours))

rotated_qx_their[3, 3, 1, :], rotated_qx_ours[3, 3, 1, :]

torch.Size([16, 4]) torch.Size([4, 16, 4, 4])
torch.Size([16, 4]) torch.Size([4, 16, 4, 8]) torch.Size([4, 16, 4, 8])
torch.Size([16, 4]) torch.Size([4, 16, 4, 8]) torch.Size([4, 16, 4, 8])
True


(tensor([ 4.6672, -0.7728, -1.5132,  2.0685,  2.6615,  1.2631, -5.4974, -0.7179]),
 tensor([ 4.6672, -0.7728, -1.5132,  2.0685,  2.6615,  1.2631, -5.4974, -0.7179]))